# Pipeline to create the personalized Boolean Networks based on gene expression and Copy Number Variation Data 

In [2]:
import pandas as pd
import os
import shutil

from functions.generate_utils.identification_patients.get_patients_sens_res import get_patients

from functions.generate_utils.identification_drug.drug_analysis import identify_drug

# from functions.generate_utils.create_generic_models.update_nodes_names import replace_node_names_in_file


from functions.generate_utils.create_generic_models.update_nodes_names import replace_node_names_in_file


from functions.analysis_utils.MaBoSS_simulation.maboss_phenotype_patient import compute_phenotype_table, collect_group_data

from functions.analysis_utils.stats.stats_proba import compute_mannwhitneyu_test_means
from functions.analysis_utils.results_MaBoSS_visualization.boxplot_phenotype import create_boxplot


from functions.generate_models import generate_models_re, pre_process_re


### Loading the datasets 

In [3]:
# Loading the data (TCGA, cell model passport)

annotations_models = pd.read_csv('data/model_list_20250407.csv')
mutations_data = pd.read_csv('data/mutations_all_20250318.csv')
drug_data = pd.read_csv('data/drug_sensitivity.csv')

montagud_data = (
    pd.read_csv('data/Montagud_inter_nodes_data.csv', header=1)
    .loc[:, ['Target node', 'Interaction type', 'Source']])
rna_seq_data = pd.read_csv('data/rnaseq_merged/rnaseq_merged_20250117.csv')
cnv_data = pd.read_csv('data/cellmodel_data/cnv_summary_20250207.csv')

proteins_data = pd.read_csv('data/cellmodel_data/proteomics_all_20250211.csv')


models_depmap_annotation = pd.read_csv('data/cellmodel_data/model_list_20250423.csv')
mutations_data_depseq = pd.read_csv('data/depmap_data/OmicsSomaticMutations.csv')


/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_96911/496007439.py:17: DtypeWarning: Columns (22,50,56,57,58,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations_data_depseq = pd.read_csv('data/depmap_data/OmicsSomaticMutations.csv')


In [5]:
# identify_drug(drug_data, annotations_models, tissue_remove)

### Users parameters data 

In [6]:
type_models = 'proteins_models'


In [7]:
# User's data to choose
number_patients = 35 # in each group
drug_interest = 'Pictilisib'
drug_targets = ['PI3K']

# drug_interest = 'Pictilisib'

# drug_targets = ['PI3K']

tissue_interest = 'PAN_CANCER'
tissue_remove = 'Haematopoietic and Lymphoid'



# Models parameters
node_to_remove = ['FUSED_EVENT', 'AR_ERG']



# Generic Boolean Network 
# Inputs
inputs_list = ['EGF', 'FGF', 'TGFB', 'ANDROGEN', 'HYPOXIA', 'NUTRIENTS', 'CARCINOGEN', 'ACIDOSIS', 'TNF', 'SPOP']

# Phenotypes
phenotype_interest = ["PROLIFERATION","INVASION","DNA_REPAIR","MIGRATION","APOPTOSIS"]



# to specify the boolean networks to be genes or proteins based
name_montagud_maps = {
        "CASPASE8": "CASP8",
        "CASPASE3": "CASP3",
        "CASPASE9": "CASP9",
        "CYCLINB": "CCNB1",
        "CYCLIND": "CCND1",
        "DSH": "DVL1",
        "BETA_CATENIN": "CTNNB1",
        "E_CADHERIN": "CDH1",
        "CYCC": "CYCS",
        "MEK1_2": "MEK1",
        "NF_KB": "NFKB",
        "SNAIL": "SNAI1",
        "TNFALPHA": "TNF",
        "TSC1_2": "TSC1",
        "BCL_XL": "BCL2L1",
        "MAP3K1_3": "MAP3K1",
        "CHK1_2": "CHK1",
    }

nodes_to_add = ['MEK2', 'TSC2', 'MAP3K3', 'CHK2']



# proteins synonyms 
synonyms_maps = {
        "RAF1": "RAF",
        "BCL2L2": "BCL2",
        "SMAD1": "SMAD",
        "SMAD3": "SMAD",
        "SMAD4": "SMAD",
        "SMAD5": "SMAD",
        "BAK1": "BAK",
        "GSK3A": "GSK3",
        "GSK3B": "GSK3",
        "TGFBR2": "TGFBR",
        "TGFBR3": "TGFBR",
        "NRAS": "RAS",
        "KRAS": "RAS",
        "HRAS": "RAS",
        "AKT2": "AKT",
        "MYCN": "MYC",
        }


In [8]:
# Create folders structure 
#subdir = f"{drug_targets[0]}_{drug_targets[1]}_target_block"
subdir = f"{'_'.join(drug_targets)}_target_block"



folder_generic_models = f"analysis/{drug_interest}/{type_models}/{subdir}/models/generic/"
folder_models = f"analysis/{drug_interest}/{type_models}/{subdir}/models"



patients_categ = ['resistant', 'sensitive']

if not os.path.exists(f"analysis/{drug_interest}/{type_models}/{subdir}/models/generic"):
        os.makedirs(f"analysis/{drug_interest}/{type_models}/{subdir}/models/generic")

for patient_categ in patients_categ:
    if not os.path.exists(f"analysis/{drug_interest}/{type_models}/{subdir}/results/{patient_categ}"):
        os.makedirs(f"analysis/{drug_interest}/{type_models}/{subdir}/results/{patient_categ}")

    if not os.path.exists(f"analysis/{drug_interest}/{type_models}/{subdir}/models/{patient_categ}"):
        os.makedirs(f"analysis/{drug_interest}/{type_models}/{subdir}/models/{patient_categ}")



dest_dir = f"analysis/{drug_interest}/{type_models}/{subdir}/models/generic"
os.makedirs(dest_dir, exist_ok=True)

# Copy the files
shutil.copy('data/montagud_models/Montagud2022_Prostate_Cancer_original.bnd', dest_dir)
shutil.copy('data/montagud_models/Montagud2022_Prostate_Cancer_original.cfg', dest_dir)


'analysis/Pictilisib/proteins_models/PI3K_target_block/models/generic/Montagud2022_Prostate_Cancer_original.cfg'

### Pre-processing the data (genes, cnv)

In [9]:
# Pre process data and saved them in the data filtered folder
# AS tissue interest is PAN cancer -> tissue_interest = None (want all tissues)

top_resistant_ids, top_sensitive_ids, top_healthy_ids, montagud_nodes, rna_seq_data_filtered, cnv_data_filtered, table_rna_seq_patients, df_melted_protein, table_proteins_patients = pre_process_re(
    montagud_data,
    rna_seq_data,
    cnv_data,
    number_patients,
    drug_data,
    annotations_models,
    drug_interest,
    proteins_data,
    type_models,
    name_montagud_maps,
    nodes_to_add,
    synonyms_maps,
    tissue_interest=None,
    tissue_remove=None,
    node_to_remove=None,
)


os.makedirs(f"analysis/{drug_interest}/data_filtered", exist_ok=True)
rna_seq_data_filtered.to_csv(f"analysis/{drug_interest}/data_filtered/rna_seq_data_filtered.csv")
cnv_data_filtered.to_csv(f"analysis/{drug_interest}/data_filtered/cnv_data_filtered.csv")
table_rna_seq_patients.to_csv(f"analysis/{drug_interest}/data_filtered/table_rna_seq_patients.csv")




/Users/romane/repos/icr/Attractor-Resistance/functions/generate_utils/identification_patients/get_patients_sens_res.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations_models_filtered.rename(
/Users/romane/repos/icr/Attractor-Resistance/functions/generate_utils/pre_process_data/pre_process_genes.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna_seq_data.rename(columns={"gene_symbol": "gene_name"}, inplace=True)
/Users/romane/repos/icr/Attractor-Resistance/functions/generate_utils/pre_process_data/pre_process_proteins.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

### Create the personalized PAN cancer models (genes and cnv)

In [ ]:
# create the personalized models (gene expression and cnv data)
generate_models_re(
    folder_generic_models,
    folder_models,
    top_resistant_ids,
    top_sensitive_ids,
    drug_interest,
    drug_targets,
    phenotype_interest,
    rna_seq_data,
    montagud_nodes,
    table_rna_seq_patients,
    cnv_data_filtered,
    name_montagud_maps,
    type_models,
    df_melted_protein,
    table_proteins_patients,
)

All .cfg and .bnd files created for sensitive and resistant patients.
patient_id: SIDM00796
🔍 Processing patient SIDM00796, gene: PI3K
PI3K node found. Replacing...
SIDM00796: CNV — nodes modified
patient_id: SIDM00834
🔍 Processing patient SIDM00834, gene: PI3K
PI3K node found. Replacing...
SIDM00834: CNV — nodes modified
patient_id: SIDM00713
🔍 Processing patient SIDM00713, gene: PI3K
PI3K node found. Replacing...
SIDM00713: CNV — nodes modified
patient_id: SIDM00547
🔍 Processing patient SIDM00547, gene: PI3K
PI3K node found. Replacing...
SIDM00547: CNV — nodes modified
patient_id: SIDM00282
🔍 Processing patient SIDM00282, gene: PI3K
PI3K node found. Replacing...
SIDM00282: CNV — nodes modified
patient_id: SIDM00255
🔍 Processing patient SIDM00255, gene: PI3K
PI3K node found. Replacing...
SIDM00255: CNV — nodes modified
patient_id: SIDM00524
🔍 Processing patient SIDM00524, gene: PI3K
PI3K node found. Replacing...
SIDM00524: CNV — nodes modified
patient_id: SIDM00763
🔍 Processing patien

In [ ]:
# check no intersection between the two groups
intersection = set(top_resistant_ids) & set(top_sensitive_ids)
print(intersection)

set()


let's check why does not tailor with protein expression 